<a href="https://colab.research.google.com/github/yopiazani02/Deep-Learning/blob/master/Youtube_comment_Model_Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Dataset

In [0]:
import json
import unicodedata
import re
# from pandas import DataFrame
import pandas as pd

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#datasets komentar youtube pada video Update Corona 22 Maret: 514 Kasus, 48 Meninggal, 29 Sembuh
dataset = pd.read_csv('/content/drive/My Drive/ytb_comment.csv')

In [0]:
# data
dataset

# Text Preprocessing

In [0]:
pip install contractions

     |████████████████████████████████| 245kB 3.2MB/s 
     |████████████████████████████████| 317kB 12.4MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81692 sha256=5e43bc0d84fc9863bd8b258aa69a628111419a8c38694f2935975eccd329f483
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [0]:
pip install sastrawi

     |████████████████████████████████| 215kB 2.8MB/s 


In [0]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk

Masukkan stopwords_id.txt kedalam folder

`/root/nltk_data/corpora/stopwords`

kemudian rename menjadi stopwords_id (hilangkan extention txt)

In [0]:
def removeStopword(str):
    stop_words = set(stopwords.words('stopwords_id'))
    word_tokens = word_tokenize(str)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)

def stemming(str):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    return stemmer.stem(str)

def cleaning(str):
    #remove non-ascii
    str = unicodedata.normalize('NFKD', str).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    #remove URLs
    str = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', str)
    #remove punctuations
    str = re.sub(r'[^\w]|_',' ',str)
    #remove digit from string
    str = re.sub("\S*\d\S*", " ", str).strip()
    #remove digit or numbers
    str = re.sub(r"\b\d+\b", " ", str)
    #to lowercase
    str = str.lower()
    #Remove additional white spaces
    str = re.sub('[\s]+', ' ', str)
    # print(str)   
    return str

def slangWord(str):
  # str = str.lower()
  # file = '/slang.dic'
  file = '/content/drive/My Drive/Colab Notebooks/slang.dic'
  fix2 = {}
  with open(file) as f:
    for line in f:
       (key, val) = line.split(':')
       fix2[key] = val.replace('\n',' ')
  tweet_t = nltk.word_tokenize(str)
  clean = " "
  for i in tweet_t:
    for slang, formal in fix2.items():
        if i == slang:
            i = formal
    if i.isalnum():
        clean = clean+i+' '
        # clean = clean+i
        # clean = ' '.join(clean)
    else:
        clean = clean[:-1]+' '+i+' '
        # clean = clean[:-1]+i
        # clean = ' '.join(clean)
  str = clean
  # print(str)
  return str

def preprocessing(str):
    # print(str)
    
    str = cleaning(str)
    str = slangWord(str)        
    str = removeStopword(str)
    str = stemming(str)
    
    return str

In [0]:
# comment = dataset['Comment']
comment = dataset['Comment'][:10]

In [0]:
comment

In [0]:
data_clean = []
data_text = dataset['Comment']
for item in data_text:
  tmp = preprocessing(item)
  data_clean.append(tmp)

dataset['Comment'] = data_clean

In [0]:
dataset

In [0]:
# Buat fungsi sentiment analysis menggunakan TextBlob
from textblob import TextBlob
import re
import numpy as np
import requests
import time

def analize_sentiment(comment):
  # print(comment)
  try:
    analysis = TextBlob(str(comment))
    time.sleep(0.2)
    toEN = analysis.translate(to='en')
    time.sleep(0.2)
    if toEN.sentiment.polarity > 0:
      return('positif')
    elif toEN.sentiment.polarity == 0:
      return('netral')
    else:
      return('negatif')
  except:
    return ('netral')

In [0]:
# Tambahkan kolom label ke dalam dataframe:
dataset['label'] = np.array([analize_sentiment(comment) for comment in dataset['Comment'] ])

# Tampilkan data:
display(dataset)

In [0]:
pos_tweets = [ tweet for index, tweet in enumerate(dataset['Comment']) if dataset['label'][index] == 'positif']
neu_tweets = [ tweet for index, tweet in enumerate(dataset['Comment']) if dataset['label'][index] == 'netral']
neg_tweets = [ tweet for index, tweet in enumerate(dataset['Comment']) if dataset['label'][index] == 'negatif']

In [66]:
total_pos = len(pos_tweets)*100/len(dataset['Comment'])
total_net = len(neu_tweets)*100/len(dataset['Comment'])
total_neg = len(neg_tweets)*100/len(dataset['Comment'])

print("Total Positif: ", total_pos)
print("Total Netral: ", total_net)
print("Total Negatif: ", total_neg)

Total Positif:  0.0
Total Netral:  100.0
Total Negatif:  0.0


In [0]:
#menghapus kolom username pada datasets
dataset = dataset.drop(columns="Username")
dataset

In [0]:
#export data to CSV
dataset.to_csv('/content/drive/My Drive/clean_data1.csv', index = False)